In [1]:
# Import necessary libraries
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


Using TensorFlow backend.


In [2]:
%matplotlib inline

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from scipy import misc

import os
import random
import pickle

In [3]:
#know which Keras library is being used Theano or Tenserflow
print "using ordering:", K.image_dim_ordering()

using ordering: tf


In [5]:
imageFolder = "-images"

image_dim = 3 #RGB

imgs = {}

for fileName in ["Cairo_North_2000.jpg", "Cairo_North_2005.jpg", "Cairo_North_2010.jpg", "Cairo_North_2016.jpg"]:
    img = misc.imread("/".join([imageFolder, fileName]))

    if image_dim == 1 and len(img.shape) > 2: 
        img = img[:,:,0]
        
    img = img / 255.0
    imgs[fileName] = img

print "Load data complete"

MemoryError: 

In [ ]:
img = imgs["Cairo_North_2000.jpg"]
print "image dimensions:", img.shape

imshow(img, cmap = plt.get_cmap('gray'), vmin = 0, vmax = 1,  interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
import math

targetRes = 50

img = imgs["Cairo_North_2000.jpg"] #do for each image?is this the target image? change it then
xStep = int( math.floor( float(img.shape[0]) / targetRes ) )
yStep = int( math.floor( float(img.shape[1]) / targetRes ) )

data = []

for y in range(yStep):
    for x in range(xStep):
        
        sample = []
        
        crop = imgs["Cairo_North_2000.jpg"][x * targetRes : (x + 1) * targetRes, y * targetRes : (y + 1) * targetRes]
        sample.append(crop)

        for layer in ["Cairo_North_2000.jpg", "Cairo_North_2005.jpg", "Cairo_North_2010.jpg", "Cairo_North_2016.jpg"]:
            target = imgs[layer][x * targetRes : (x + 1) * targetRes, y * targetRes : (y + 1) * targetRes]
            target_val = int ( round( np.mean(target) ) )
            sample.append(target_val)

        data.append(sample)

In [ ]:
img_index = 100

img = data[img_index][0]
print "image dimensions:", img.shape
print "Value at time 0:", (data[img_index][1])
print "Value at time 1:", (data[img_index][2])
print "Value at time 2:", (data[img_index][3])
print "Value at time 3:", (data[img_index][4])

imshow(img, cmap = plt.get_cmap('gray'), vmin = 0, vmax = 1,  interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
recomp = {}

for i, layer in enumerate(["2000.jpg", "2005.jpg", "2010.jpg", "2016.jpg"]):

    recomp[layer] = np.ndarray((targetRes * xStep, targetRes * yStep), dtype=np.float32)

    for y in range(yStep):
        for x in range(xStep):
            recomp[layer][x * targetRes : (x + 1) * targetRes, y * targetRes : (y + 1) * targetRes] = data[ y * xStep + x ][i+1]

        
for layer in ["2000.jpg", "2005.jpg", "2010.jpg", "2016.jpg"]:
    
    print layer
    imshow(recomp[layer], cmap = plt.get_cmap('gray'), vmin = 0, vmax = 1,  interpolation='nearest')
    plt.axis('off')
    plt.show()

In [ ]:
# number of classes
num_classes = 2 #reducing number of classes 

# image dimensions
img_rows, img_cols = X_train.shape[1],  X_train.shape[2]

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

print X_train.shape
print y_train.shape

In [ ]:
# model hyperparameters
batch_size = 128 # *increasing the number of training examples in one forward/backward pass--> making it too large can
# cause the kernel to keep dying 
nb_epoch = 30

# network architecture
patch_size_1 = 4 #reducing the patch size for deeper neural network/more extensive connections
patch_size_2 = 4
patch_size_3 = 4 

depth_1 = 40
depth_2 = 50 #increasing depth size, but not too much to the hundreds 
depth_3 = 50

pool_size = 2

num_hidden_1 = 100
num_hidden_2 = 100

dropout = 0.3

In [ ]:
# create new Keras Sequential model
model = Sequential()

# add first convolutional layer to model and specify it's depth and filter size
# for the first layer we also have to specify the size of each input image
# which we calculated above
model.add(Convolution2D(depth_1, patch_size_1, patch_size_1,
                        border_mode='valid',
                        input_shape=input_shape))
# apply 'relu' activation function for first layer
model.add(Activation('relu'))
# apply max pooling to reduce the size of the image by a factor of 2
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

# repeat these operations for the second convolutional layer
# this time Keras can figure out the input size 
# from the previous layer on it's own
model.add(Convolution2D(depth_2, patch_size_2, patch_size_2,
                        border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(depth_3, patch_size_3, patch_size_3,
                        border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))


# flatten the three-dimensional convolutional layer to a single layer of neurons
model.add(Flatten())

# add the first fully connected layer, applying 'relu' activation and dropout
model.add(Dense(num_hidden_1))
model.add(Activation('relu'))
model.add(Dropout(dropout))

# add the second fully connected layer
model.add(Dense(num_hidden_2))
model.add(Activation('relu'))
model.add(Dropout(dropout))

# add the final classification layer with the number of neurons 
# matching the number of classes we are trying to learn
model.add(Dense(num_classes))

# apply the 'softmax' activation to the final layer to convert the output to 
# a probability distribution
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)

print 'Test score:', score[0]
print 'Test accuracy: {:.2%}'.format(score[1])